<a href="https://colab.research.google.com/github/xRAFPx/AAII/blob/master/Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

In [0]:
import requests
import json
import csv
import string
import io
import pandas as pd
from tempfile import NamedTemporaryFile
import shutil
from datetime import date
import datetime
import os.path
from os import path

In [0]:
def countDiffDates(lastDay, cityCSV, lastData):

    #print(type(today))

    cw = open(cityCSV, 'r')
    listCity = list(cw)
    dateList = list()
    for row in listCity:
        date = row.split(',')[0]
        dateList.append(date)
    cw.close()  

    date_objectNew = datetime.datetime.strptime(lastDay, '%Y-%m-%d').date()
    date_object = datetime.datetime.strptime(dateList[-1], '%Y-%m-%d').date()

    diff = date_objectNew - date_object
    diffDays = diff.days
    print("Days Missing:", diffDays)
    if diffDays != 0:
        res = lastData[-diffDays:]
        print(res)
        # # Open file in append mode
        # with open(cityCSV, 'a', newline='') as write_obj:
        #     # Create a writer object from csv module
        #     csv_writer = csv.writer(write_obj, dialect='excel', delimiter=',', quoting=csv.QUOTE_NONE, quotechar='', escapechar=' ')
        #     # Add contents of list as last row in the csv file
        #     csv_writer.writerow(res)
        
        f = open(cityCSV, 'a', newline='')
        for x in res:
          f.write(x) #Give your csv text here.
          ## Python will convert \n to os.linesep
        f.close()
        print(res)
    else:
      print("Data is Updated")

In [0]:
def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

In [0]:
# response = requests.get("https://api.ipma.pt/open-data/distrits-islands.json")

# print(response.status_code)
# jprint(response.json())

In [0]:
def getDico():
  response = requests.get("https://api.ipma.pt/open-data/distrits-islands.json")
  local = ['Porto', 'Lisboa', 'Faro', 'Evora', 'Coimbra']
  dico = list()

  for x in response.json()['data']:
      for l in local:
          if x['local'] == l:
              dico.append([l.lower(),"{0:0=2d}".format(int(x['idDistrito'])) + str("{0:0=2d}".format(int(x['idConcelho'])))])
  return dico

# ![CCDR Regions](https://upload.wikimedia.org/wikipedia/commons/thumb/c/c7/Area_atua%C3%A7%C3%A3o_CCDR.png/800px-Area_atua%C3%A7%C3%A3o_CCDR.png)

In [0]:
def getIPMAData(dico):
  dictURL = {1: ['temperature-min','mtnmn'], 2: ['temperature-max', 'mtxmx']} 
  today = date.today()
  for d in range(len(dico)):
      print("_______________\n" + str(dico[d][0]).upper() + "\n_______________")
      for i in dictURL:
          #print(dictURL[i])
          url = 'https://api.ipma.pt/open-data/observation/climate/' + dictURL[i][0] + '/' + dico[d][0] + '/' + dictURL[i][1] + '-' + dico[d][1] + '-' + dico[d][0] + '.csv'
          print('\n' + str(dictURL[i][0]) + '\n' + url) 
          folder = '/content/drive/My Drive/Colab Notebooks/AII Project'
          cityCSV = folder + '/' + str(dico[d][0]) + '_' + dictURL[i][0] + '.csv' 
          cityCSV_data = folder + '/Daily Data/' + str(today) + '_' + str(dico[d][0]) + '_' + dictURL[i][0] + '.csv' 
          print(cityCSV)
          with requests.Session() as s:
              download = s.get(url)
              print("Request Status:", download.status_code)
              decoded_content = download.content.decode('utf-8')
  #            #print(decoded_content)
              cr = csv.reader(decoded_content.splitlines(), delimiter=',')
              my_list = list(cr)
              lastDay = my_list[-1][0]
              if path.exists(cityCSV):
                print("Original file already exists")
                print("Creating validation file for ", str(today))
                with open(cityCSV_data, mode = 'w', newline='') as result_file:
                  wrD = csv.writer(result_file, dialect='excel', delimiter=',', quoting=csv.QUOTE_NONE, quotechar='', escapechar=' ')
                  wrD.writerows(my_list)
              else:
                with open(cityCSV, mode = 'w', newline='') as result_file:
                  wr = csv.writer(result_file, dialect='excel', delimiter=',', quoting=csv.QUOTE_NONE, quotechar='', escapechar=' ')
                  wr.writerows(my_list)
              countDiffDates(lastDay, cityCSV, my_list)

In [0]:
dico = list()
dico = getDico()

In [41]:
getIPMAData(dico)

_______________
COIMBRA
_______________

temperature-min
https://api.ipma.pt/open-data/observation/climate/temperature-min/coimbra/mtnmn-0603-coimbra.csv
/content/drive/My Drive/Colab Notebooks/AII Project/coimbra_temperature-min.csv
Request Status: 200
Original file already exists
Creating validation file for  2020-04-28
Days Missing: 0
Data is Updated

temperature-max
https://api.ipma.pt/open-data/observation/climate/temperature-max/coimbra/mtxmx-0603-coimbra.csv
/content/drive/My Drive/Colab Notebooks/AII Project/coimbra_temperature-max.csv
Request Status: 200
Original file already exists
Creating validation file for  2020-04-28
Days Missing: 0
Data is Updated
_______________
FARO
_______________

temperature-min
https://api.ipma.pt/open-data/observation/climate/temperature-min/faro/mtnmn-0805-faro.csv
/content/drive/My Drive/Colab Notebooks/AII Project/faro_temperature-min.csv
Request Status: 200
Original file already exists
Creating validation file for  2020-04-28
Days Missing: 0
